In [51]:
import urllib
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import regex as re
from tqdm.notebook import tqdm
import os

site = 'https://kia-rio.net/forum/'
folder_names = [
    ['0', 'Отзывы', 'f=117', 'KIA RIO'],
    ['1', 'Двигатель', 'f=53', 'KIA RIO'],
    ['2', 'Трансмиссия', 'f=51', 'KIA RIO'],
    ['3', 'Ходовая', 'f=75', 'KIA RIO'],
    ['4', 'Колеса', 'f=118', 'KIA RIO'],
    ['5', 'Электрооборудование', 'f=52', 'KIA RIO'],
    ['6', 'Мультимедиа', 'f=64', 'KIA RIO'],
    ['7', 'Кузов', 'f=83', 'KIA RIO'],
    ['8', 'Жидкости', '', 'KIA RIO'],
    ['9', 'Сервис', 'f=19', 'KIA RIO'],
    ['10', 'Эксплуатация', '', 'KIA RIO'],
    ['11', 'Тюнинг', 'f=84', 'KIA RIO'],
    ['12', 'Двигатель_система_охлаждения', 'f=59', 'KIA RIO'],
    ['13', 'Кузов_кондиционер', 'f=74', 'KIA RIO'],
    ['14', 'Сервис_ремонт', 'f=20', 'KIA RIO'],
]

На странице раздела у нас две задачи:  
1. Найти и сохранить все ссылки на темы
2. Найти ссылку на следующую страницу раздела или убедиться, что это последняя страница раздела.

In [52]:
def collect_themes(site, folder):
    
    site_address = site + 'forumdisplay.php?' + folder
    themes = []
    flag = True

    while flag:
    
        resp = urlopen(site_address) # скачиваем файл
        html = resp.read().decode('utf-8') # считываем содержимое
        soup = BeautifulSoup(html, 'html.parser') # делаем суп
    
        flag = False
        for link in soup.find_all('a'):
            if link.has_attr('href') and link.has_attr('id'):
                if re.search('thread_title_', str(link.get('id'))):
                    s = link.get('href')
                    s = site + 'showthread.php?' + re.search('t=[0-9]+', s)[0]
                    themes.append([link.get_text(), s])
            if not flag and link.has_attr('href') and link.has_attr('rel'):
                if link.get('rel')[0] == 'next':
                    flag = True
                    s = link.get('href')
                    site_address = site + 'forumdisplay.php?' + re.search(folder + '[\S]+', s)[0]
    return themes

На странице темы у нас три задачи:  
1. Понять, что являеся шапкой темы (это сообщение #1), прочитать ее только один раз - НЕ СДЕЛАНО
2. Прочитать и сохранить все сообщения (кроме шапки) на странице темы
3. Найти ссылку на следующую страницу темы или убедиться, что это последняя страница темы.

In [53]:
def collect_messages(site, themes):

    messages = []
    for theme in tqdm(themes):
        
        print(theme[0])
        theme_address = theme[1]
        thread = re.search('t=[0-9]+', theme_address)[0]
        flag = True
        theme_messages = []
        post_data = {}

        while flag:
            resp = urlopen(theme_address) # скачиваем файл
            html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
            soup = BeautifulSoup(html, 'html.parser') # делаем суп
            #Собираем сами сообщения
            divs = soup.find_all('div')
            for div in divs:
                if div.has_attr('id') and re.search('(postmenu_)([0-9]+)(_menu)', div['id']):
                    continue
                if div.has_attr('id') and re.search('postmenu_', div['id']):
                    author = div.find('a')
                    if author != None:
                        author_id = re.search('u=[0-9]+', author['href'])[0]
                        author_nickname = author.get_text()
                    else:
                        author_id = div.get_text().strip()
                        author_nickname = 'Guest'
                    post_data[re.sub('postmenu_', '', div['id'])] = [author_id, author_nickname]
                if div.has_attr('id') and re.search('post_message_', div['id']):
                    message_id = re.sub('post_message_', '', div['id'])
                    if message_id in post_data:
                        author = post_data[message_id][1]
                    else:
                        author = ''
                    for item in div.children:
                        if item.name == 'div':
                            if item.has_attr('class') and item['class'][0] == 'smallfont':
                                post_date = item.get_text().strip()
                            item.clear()
                    s = re.sub('[\']+', '`', div.get_text())
                    s = '\'' + re.sub('[\s]+', ' ', s).strip() + '\''
                    if len(s) > 2:
                        theme_messages.append([post_date, author, s])
            # В отдельном цикле ищем ссылку на следующую страницу темы (это 'a' с атрибутом 'rel', равным 'next')
            # Если ссылка найдена, 
            flag = False 
            for link in soup.find_all('a'):
                if not flag and link.has_attr('href') and link.has_attr('rel'):
                    if link.get('rel')[0] == 'next':
                        flag = True
                        s = link.get('href')
                        theme_address = site + 'showthread.php?' + re.search(thread + '[\S]+', s)[0]
        messages.append(['\''+ theme[0] + '\'', theme_messages])
        
    return messages

In [54]:
def save_messages(messages, folder_name):
    f_name = os.path.join(folder_name[3], folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv')
    with open(f_name, 'w', encoding='utf-8') as ouf:
        ouf.write('Code,Folder,Theme,Author,Date,Message\n')
        for message in messages:
            header = folder_name[:2] + [message[0]]
            for item in message[1]:
                ouf.write(','.join(header + item))
                ouf.write('\n')

In [55]:
def scraper(site, folder_names):
    for folder_name in folder_names:
        if len(folder_name[2]) > 0:
            folder = folder_name[2]
            current_themes = collect_themes(site, folder)
            current_messages = collect_messages(site, current_themes)
            save_messages(current_messages, folder_name)
    print('Information collected')

In [56]:
%load_ext ipytelegram
%reload_ext ipytelegram
import telepot
bot = telepot.Bot('1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A')
response = bot.getUpdates()
%telegram_setup 1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A 1305740495

The ipytelegram extension is already loaded. To reload it, use:
  %reload_ext ipytelegram


In [57]:
%%telegram_send Next message
scraper(site, folder_names)
'information collected???'

Отзыв владельца Киа Рио 2020 1,6 Премиум
Отзыв владельца Киа Рио 4 X-Line после Киа Рио 3
Отзыв владельца Киа Рио 2020 Премиум
Все "косяки" Киа Рио 2017-2020 - отзывы владельцев
Владельцы комплектации Premium, к Вам есть вопросы
Киа Рио X-Line как второй автомобиль в семье
Выбор пал на Киа
Как я повстречал свою Корейскую морковку
Помогу с покупкой по хорошей цене нового Киа Рио и других автомобилей
Совет по покупке б-у автомобиля
Отзыв владельца KIA RIO IV
Приобрел Киа Рио Премиум
Отзыв Киа Рио 2019 Premium
Скидки и подарки при покупке KIA RIO 4 2017
Отзыв владельца Киа Рио 2019: в сомнениях
Отзыв Киа Рио 2019 Престиж
Отзыв KIA Rio 2019. 1.6 AT. Premium. Черныш
Движок Карра 1,4+ 6 ст. механика.
Киа Рио 2017 новый кузов хэтчбек
Рестайлинг Киа Рио 2019
Киа Рио 2018 или Skoda Rapid 2018
Личные впечатления от нового Киа Рио 2017-2018
Отзыв Киа Рио Luxe 1,6 AT Luxe после Solaris
Отзыв владельца Киа Рио 2017 Premium (новый кузов)
Отзыв владельца Киа Рио 2018 Premium (новый кузов)
Отзыв Киа Р

Смена вязкости масла после 100 000км
Вибрация в новом Киа Рио на холостых оборотах
Удаление катализатора
Сброс среднего расхода после каждой заправки
Какой бензин вы заливаете в КИА РИО 2017-2020?
Воздушный фильтр двигателя
Провал при разгоне 1,6 мкпп
Прогрев 1,4
Медленный разгон
Регулировка клапанов
Замена масла в двигателе Рио 4 1.6 АТ
Скрип/свист под капотом
Катализатор: есть ли проблемы, удалять или не удалять
Свечи зажигания
Масло между КПП и двигателем
Расход на Рио 4 Автомат 1.4. (30 км. с горящей лампой по городу)
Запуск двигателя не с первого раза 1,6 x line 2018 года выпуска
Уровень масла двигателя: как замерять правильно?
Чип-тюнинг Киа Рио 4
Воздушный фильтр абсорбера топливного бака
Высокие обороты при запуске двигателя
Масляный фильтр
Пищит при движении
Кио Рио 4 , именно 1.4 двигатель. Там гидрокомпенсаторы или толкатели?
Самостоятельная замена масла
Капля масла между двигателем и коробкой
Расход топлива
Стук при запуске холодного двигателя
Машина не тянет и глохнет
Трес

Щелчёк при начале движения. АКПП
Акпп замена жидкости на 90т. км?!
АКПП 6 Рио 2020 динамика разгона
Киа Рио 4. Вой МКПП на 5-ой передаче
Сливная пробка на АКПП
Стук при трогании с места
Блокировка акпп
Сломалась пружина педали сцепления
А можно ли вывести на дисплей номер передачи АКПП?
Нечеткое переключение передач. 6 МКПП
Буксует сцепление кия Рио х Лайн 1.6
Разные обороты двигателя при движении с определенной скоростью
Кикдаун на АКПП
Толчки при переключениях передач АКПП
Температура АКПП 6ст 2019
механическая кпп вылетает 2 передача
Управление АКПП: вопросы новичка
АКПП  не выходит из P
Звук при отжиме сцепления
Пинок АКПП при сбросе скорости
Буксует сцепление
Вой АКПП
АКПП и внешние воздействия при ДТП
уровень ATF
Проскальзывает сцепление (обороты растут, а скорость нет)
Переключение МКПП клацанье, хруст
Внешний пыльник шруса Kia Rio IV X-Line
Странности переключения передач на МКПП
Выдавило сальник на коробке передач
Стук при включении передач
Щелчок на селекторе при отпускании п

Дребезг (подобно банке с болтами) спереди справа на неровностях
Глюк  системы стабилизации  и звуки от блока АБС
Замена тормозной жидкости с ABS/ESC
Отказ тормозов из-за лопнувшего тормозного шланга
Добраться до заднеступичного
Икс лайн - пришло время подвески
Работа стоек на новой Киа рио 4 поколения
Тормозные колодки Киа Рио 2017-2020
Бряканье рулевой колонки
Тормозные диски и барабаны Киа Рио 2017-2020
Стук при повороте рулевого колеса на месте
Скрипящий звук сзади слева
Подвеска соляриса и рио
Схождение задних колес
Рио 2018 года мкпп не едет совсем
Щелчок при повороте руля
Кривой руль, тянет вправо
Задние барабаные тормоза - свист при нажатии педали
Стойки стабилизатора
Скрежет при полной остановки
Странное поведение тормозов при резком нажатии
Кидает машину на трассе
Стук в районе переднего левого колеса
Тормозная жидкость
"трущийся" звук при повороте рулевого колеса
При нажатии на тормоз пищит правое переднее колесо
Левый привод
Стояночный тормоз - регулировка
Как ведет себя ЭУР

Колесные гайки
Давление в шинах разница ?
Что за диски?
Зимняя резина: ваш выбор
Замена штатной летней резины
Параметры дисков Киа Рио 2017-2020
Увеличить размер - шины 205/65R15
Пробил заднее колесо
Штампованные диски под штатные колпаки Rio X-Line 2019 года
Шины 205/55 R16
Секретки на Kia Rio 2017 FB
Горит лампа  TPMS
Литые диски реплика Replay
Покупка зимней шипованной резины в Сочи
Замена оригинальных литых дисков на реплику
Шины 195/55 R16
Колпаки
Неправильно поставили колеса на шиномонтаже?
Баллонный ключ
Гайки крепления колес Kia Rio Premium 2017



...

Каталог запчастей Киа Рио 2017-2020
Автоматическое складывание зеркал
Вкусняшки от Али
Органайзер с фальш-полом в Рио Х-лайн
Размеры багажника
Дисплей парктроника
Уплотнитель междверный РКИ-19
дефлекторы на КИА РИО 2017 г
Новая Rio 2020, рестайлинг. Подбор запчастей и расходников по VIN
Замена штатной антенны на "плавник"
Стоимость допов при покупке
Подлокотник
Самостоятельная установка сетки в решетку радиатора на рио-2017
Система контроля давления в шинах
Парктроник
Накладка на низ переднего бампера
ТСУ оно же прицепное.пос
Kia Rio 4 спойлер( кто может быть заказывал?)
Лип спойлер. Где приобрести. Артикульный номер
Сканер аналогичный дилерскому
Декоративная крышка двигателя Kia Rio 1.4 Kappa
Каталог запчастей Киа Рио 2017-2020
Брызговики на седан RIO-4
Накладки порогов и заднего бампера
Автоодеяло на Kia Rio 2018
Постепенное оснащение от Комфорт до Премиум
Органайзер в запаску для домкрата
Замена ручек блока управления отопителем
Подключение видеорегистратора к плафону освещения.
Сет

Розовый антифриз Coolstream A-110 вместо зеленого
Антифриз
Падает температура на холостых.
Идёт жар в ноги
В пробке воздух в системе вентиляции салона становится теплее
Включение вентилятора на трассе зимой.



Салонный фильтр Киа Рио 2017-2020
Включение рециркуляции при омывании лобового стекла
Шум при включении компрессора кондиционера
Не греется на холостых.
Поток воздуха из закрытых дефлекторов
Дует из дефлекторов при выключенном климате
Климат-контроль - слишком холодно!
Нужна ли замена  радиатора кондиционера
Отзывная компания по вентилятору климатической установки
Температура из дефлекторов при включение печки
Дует в колено
Температура воздуха из дефлекторов (климат-контроль)
Автоматическое включение кондиционера при обдуве на стекло?
Сильно потеют стекла
Лужа под машиной
Неприятный запаx из системы вентиляции
Рио 4. Как отключить блокировку кондиционера в зимнее время?
Не работает кондиционер
KIO RIO 2017 кондиционер-климат
Пахнет незамерзайкой при включении кондиционера
Плохо работает кондиционер



Зазор после ДТП мелкого
Покраска заднего бампера РИО
Дефекты на кузове
Процарапал левый порог в зоне антигравийного покрытия...
Рыжик вырос на крыше, где сделать, Москва
Стоимость ремонта по страховке Kia Rio 3
Дубль переднего бампера на Рио3
Поймал камень в лобовое
Трещина в переднем бампере
Ударил 2 двери
Ремонт после дтп
заглушка в переднем бампере
Крошится покрытие "черная пленка" на дверях
Где купить передний бампер а цвет?
Ремонт после удара
Где сделать кузовной ремонт с такими повреждениями?
Китайские аналоги(автозапчасти). Стоит ли приобретать?
Капот - оригинал/неоригинал/с разборки?
Зазоры после ДТП
Стоит ли покупать неоригинальные запчасти???
Кузовной ремонт сервис
Посоветуйте кузовной ремонт
Задний бампер
Дефект лкп после покраски переднего бампера
Прилетело на Новый год =)) что теперь делать
Оторвали ручку двери
Кузовной ремонт Киа Рио 2016 седан
Где и как сделать кузовной ремонт Киа Рио?
Куда отдать машину на замену бампера?
Что посоветуете оригинал/не оригинал/бу?
Где мож

'information collected???'

In [ ]:
import pandas as pd
mess = pd.read_csv('Сервис.csv', quotechar="'")
mess

In [ ]:
themes = []
flag = True

while flag:
    
    resp = urlopen(site_address) # скачиваем файл
    html = resp.read().decode('utf-8') # считываем содержимое
    soup = BeautifulSoup(html, 'html.parser') # делаем суп
    
    flag = False
    for link in soup.find_all('a'):
        if link.has_attr('href') and link.has_attr('id'):
            if re.search('thread_title_', str(link.get('id'))):
                s = link.get('href')
                s = site + 'showthread.php?' + re.search('t=[0-9]+', s)[0]
                themes.append([link.get_text(), s])
        if not flag and link.has_attr('href') and link.has_attr('rel'):
            if link.get('rel')[0] == 'next':
                flag = True
                s = link.get('href')
                site_address = site + 'forumdisplay.php?' + re.search(folder + '[\S]+', s)[0]

In [41]:
messages = []
#for theme in tqdm(themes):
theme = ['Воздушный фильтр двигателя', 'https://kia-rio.net/forum/showthread.php?t=13723']
theme_address = theme[1]
thread = re.search('t=[0-9]+', theme_address)[0]
flag = True
theme_messages = []
post_data = {}
while flag:
    resp = urlopen(theme_address) # скачиваем файл
    html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
    soup = BeautifulSoup(html, 'html.parser') # делаем суп
    #Собираем сами сообщения
    divs = soup.find_all('div')
    for div in divs:
        if div.has_attr('id') and re.search('(postmenu_)([0-9]+)(_menu)', div['id']):
            continue
        if div.has_attr('id') and re.search('postmenu_', div['id']):
            author = div.find('a')
            if author != None:
                author_id = re.search('u=[0-9]+', author['href'])[0]
                author_nickname = author.get_text()
            else:
                author_id = div.get_text().strip()
                author_nickname = 'Guest'
            post_data[re.sub('postmenu_', '', div['id'])] = [author_id, author_nickname]
        if div.has_attr('id') and re.search('post_message_', div['id']):
            message_id = re.sub('post_message_', '', div['id'])
            if message_id in post_data:
                author = post_data[message_id][1]
            else:
                author = ''
            for item in div.children:
                if item.name == 'div':
                    if item.has_attr('class') and item['class'][0] == 'smallfont':
                        post_date = item.get_text().strip()
                    item.clear()
            s = re.sub('[\']+', '`', div.get_text())
            s = '\'' + re.sub('[\s]+', ' ', s) + '\''
            if len(s) > 0:
                theme_messages.append([post_date, author, s])
    # В отдельном цикле ищем ссылку на следующую страницу темы (это 'a' с атрибутом 'rel', равным 'next')
    # Если ссылка найдена, 
    flag = False 
    for link in soup.find_all('a'):
        if not flag and link.has_attr('href') and link.has_attr('rel'):
            if link.get('rel')[0] == 'next':
                flag = True
                s = link.get('href')
                theme_address = site + 'showthread.php?' + re.search(thread + '[\S]+', s)[0]
messages.append(['\''+ theme[0] + '\'', theme_messages])

In [42]:
messages

[["'Воздушный фильтр двигателя'",
  [['29.08.2017 15:03',
    'аdmin',
    "' Оригинальный воздушный фильтр двигателя Киа Рио 2017-2018-2019-2020: 28113H8100. Размеры: 252 х 145 х 54 мм Подходит для двигателей 1,4 и 1,6 л. Аналоги: Невский фильтр NF5003C TSN 9.1.1565 AMD AMDFA789 НЕ подходит: AP026 Filtron '"],
   ['19.12.2017 20:48',
    'Димася',
    "' Размеры оригинального 28113-H8100 : 252 х 145 х 54 мм Аналоги: TSN 9.1.1565 AMD AMDFA789 Близкие размеры у Filtron AP 026 (256х150х57) -нужно посмотреть влезет ли по высоте и ширине *Для информации: фильтр от Креты НЕ подойдёт, на ней фильтр со скосом. '"],
   ['20.12.2017 07:12',
    'artem_shubnikov',
    "' Для всех машин всегда брал аналоги от Nipparts, к сожалению не смог найти в интернете подбор их фильтра по размеру '"],
   ['10.01.2018 13:54',
    'artem_shubnikov',
    "' Кто-нибудь уже пробовал аналоги? Сейчас посмотрел-довольно грязный, завтра продую воздухом, но походу нужно менять. '"],
   ['10.01.2018 14:12',
    'kolvic

In [ ]:
f_name = os.path.join(folder_name[3], folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv')
with open(f_name, 'w', encoding='utf-8') as ouf:
    ouf.write('Code,Folder,Theme,Date,Author,Message\n')
    for message in messages:
        header = folder_name[:2] + [message[0]]
        for item in message[1]:
            ouf.write(','.join(header + item))
            ouf.write('\n')

In [ ]:
folder_name = ['0', 'Отзывы', 'f=25', 'LADA VESTA']
f_name = folder_name[3] + '/' + folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv'
f_name

In [ ]:
'''messages = []
theme_address = 'http://www.lada-vesta.net/showthread.php?t=9571'
resp = urlopen(theme_address) # скачиваем файл
html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
soup = BeautifulSoup(html, 'html.parser') # делаем суп
divs = soup.find_all('div')
for div in divs:
    if div.has_attr('id'):
        if re.search('post_message_', div['id']):
            for item in div.children:
                if item.name != 'div':
                    s = str(item).strip()
                    if len(s) > 0:
                        messages.append([s])
print(messages)'''

In [5]:
site = 'https://kia-rio.net/forum/'
folder_names = [['1', 'Двигатель', 'f=53', 'KIA RIO']]
for folder_name in folder_names:
    if len(folder_name[2]) > 0:
        folder = folder_name[2]
        current_themes = collect_themes(site, folder)
print('Information collected')

Information collected


In [6]:
current_themes

[['Смена вязкости масла после 100 000км',
  'https://kia-rio.net/forum/showthread.php?t=17089'],
 ['Вибрация в новом Киа Рио на холостых оборотах',
  'https://kia-rio.net/forum/showthread.php?t=13830'],
 ['Удаление катализатора', 'https://kia-rio.net/forum/showthread.php?t=16246'],
 ['Сброс среднего расхода после каждой заправки',
  'https://kia-rio.net/forum/showthread.php?t=17033'],
 ['Какой бензин вы заливаете в КИА РИО 2017-2020?',
  'https://kia-rio.net/forum/showthread.php?t=14520'],
 ['Воздушный фильтр двигателя',
  'https://kia-rio.net/forum/showthread.php?t=13723'],
 ['Провал при разгоне 1,6 мкпп',
  'https://kia-rio.net/forum/showthread.php?t=17035'],
 ['Прогрев 1,4', 'https://kia-rio.net/forum/showthread.php?t=15469'],
 ['Медленный разгон', 'https://kia-rio.net/forum/showthread.php?t=16760'],
 ['Регулировка клапанов', 'https://kia-rio.net/forum/showthread.php?t=17046'],
 ['Замена масла в двигателе Рио 4 1.6 АТ',
  'https://kia-rio.net/forum/showthread.php?t=16787'],
 ['Скрип